In [2]:
#load "Paket.fsx"

In [3]:
let wc = new System.Net.WebClient()
wc.DownloadFile(
    "https://github.com/DiffSharp/DiffSharp/releases/download/v0.7.7/DiffSharp-v0.7.7-Linux64.zip", 
    "/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7-Linux64.zip")

In [4]:
open System.Diagnostics

In [5]:
let ps = 
    ProcessStartInfo(
        FileName="unzip",
        Arguments="/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7-Linux64.zip -d /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7",
        RedirectStandardOutput=true,UseShellExecute=false)

In [6]:
let p = Process.Start(ps)
p.StandardOutput.ReadToEnd()

"Archive:  /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7-Linux64.zip
"

In [7]:
#I "/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/"
#r "DiffSharp"
#r "FSharp.Quotations.Evaluator"

In [8]:
let openblasFromNuget = @"/home/nbuser/IfSharp/bin/packages/OpenBLAS/lib/native/bin/x64/"
let diffSharpReleaseFromZip = @"/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/"

In [9]:
open System
Environment.SetEnvironmentVariable("LD_LIBRARY_PATH", Environment.GetEnvironmentVariable("LD_LIBRARY_PATH") + ":" + diffSharpReleaseFromZip)

In [10]:
Environment.GetEnvironmentVariable("LD_LIBRARY_PATH")

"/usr/lib64/MRO-3.3.0/R-3.3.0/lib/R/lib::/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/"

In [11]:
open DiffSharp.AD.Float64

In [14]:
let split (arr : DV) =
    let n = arr.Length
    arr.[0..n/2-1], arr.[n/2..n-1] 

let rec merge (l : DV) (r : DV) =
    let n = l.Length + r.Length
    let res = Array.zeroCreate<D> n
    let mutable i, j = 0, 0
    for k = 0 to n-1 do
        if i >= l.Length   then res.[k] <- r.[j]; j <- j + 1
        elif j >= r.Length then res.[k] <- l.[i]; i <- i + 1
        elif l.[i] < r.[j] then res.[k] <- l.[i]; i <- i + 1
        else res.[k] <- r.[j]; j <- j + 1
    toDV res
        
let rec mergeSort (arr :DV) =
    if arr.Length<2 then arr else       
    let (x, y) = split arr
    merge (mergeSort x) (mergeSort y) 

In [15]:
let rnd=System.Random()
let test= Array.init 5 (fun _-> rnd.NextDouble()) |> toDV

In [16]:
mergeSort test

DV [|0.01764531434; 0.0457408936; 0.1075394187; 0.1250395524; 0.5259685705|]

In [17]:
let jacobianmergesort=jacobian mergeSort
jacobianmergesort test

DM [[0.0; 1.0; 0.0; 0.0; 0.0]
    [0.0; 0.0; 0.0; 1.0; 0.0]
    [1.0; 0.0; 0.0; 0.0; 0.0]
    [0.0; 0.0; 0.0; 0.0; 1.0]
    [0.0; 0.0; 1.0; 0.0; 0.0]]